In [1]:
import gc
import time
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from statsmodels.robust import mad
import pywt
import scipy
from scipy.signal import butter, deconvolve
from scipy.signal import hilbert, hann, convolve
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
from tsfresh.feature_extraction import feature_calculators
from sklearn.linear_model import LinearRegression
pd.options.display.precision = 15
import xgboost as xgb
import lightgbm as lgb
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from hyperopt import STATUS_OK, tpe, hp, Trials, fmin

In [4]:
SIGNAL_LEN = 150_000
SAMPLE_RATE = 4e6

data_reader = pd.read_csv('D:/kaggle/earthquake/train.csv', 
                          dtype={'acoustic_data': np.int16,
                                 'time_to_failure': np.float32},
                          chunksize=300_000)

In [5]:
def maddest(d, axis=None):
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)


def high_pass_filter(x, low_cutoff=1000, SAMPLE_RATE=SAMPLE_RATE):
    nyquist = 0.5 * SAMPLE_RATE
    norm_low_cutoff = low_cutoff / nyquist
    
    sos = butter(10, Wn=[norm_low_cutoff], btype='highpass', output='sos')
    filtered_sig = scipy.signal.sosfilt(sos, x)
    return filtered_sig


def denoise_signal(x, wavelet='db4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode='per')
    sigma = (1/0.6745) * maddest(coeff[-level])
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')


def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]


def classic_sta_lta(x, length_sta, length_lta):
    sta = np.cumsum(x**2)
    sta = np.require(sta, dtype=np.float)
    lta = sta.copy()
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta
    sta[:length_lta - 1] = 0
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny
    return sta / lta

In [7]:
nrows = 629145481
rows = 300_000
segments = int(np.floor(nrows / rows))


X_train = pd.DataFrame(dtype=np.float64)

In [8]:
idx = 4195
for segment in tqdm(data_reader, total=segments):
    
    # ==================================================================
    # =====================original data================================
    # ==================================================================
    
    xc = pd.Series(segment.acoustic_data.values[75000:225000])
    zc = np.fft.fft(xc)
    
    X_train.loc[idx, 'mean'] = xc.mean()
    X_train.loc[idx, 'std'] =xc.std()
    X_train.loc[idx, 'max'] = xc.max()
    X_train.loc[idx, 'min'] = xc.min()   
    X_train.loc[idx, 'mean_change_abs'] = np.mean(np.diff(xc))
    X_train.loc[idx, 'mean_change_rate'] = np.mean(np.nonzero((np.diff(xc) / xc[:-1]))[0])
    X_train.loc[idx, 'abs_max'] = np.abs(xc).max()
    
    X_train.loc[idx, 'mean_first_50000'] = xc[:50000].mean()
    X_train.loc[idx, 'mean_last_50000'] = xc[-50000:].mean()
    X_train.loc[idx, 'mean_first_10000'] = xc[:10000].mean()
    X_train.loc[idx, 'mean_last_10000'] = xc[-10000:].mean()
    X_train.loc[idx, 'std_first_50000'] = xc[:50000].std()
    X_train.loc[idx, 'std_last_50000'] = xc[-50000:].std()
    X_train.loc[idx, 'std_first_10000'] = xc[:10000].std()
    X_train.loc[idx, 'std_last_10000'] = xc[-10000:].std()
    X_train.loc[idx, 'min_first_50000'] = xc[:50000].min()
    X_train.loc[idx, 'min_last_50000'] = xc[-50000:].min()
    X_train.loc[idx, 'min_first_10000'] = xc[:10000].min()
    X_train.loc[idx, 'min_last_10000'] = xc[-10000:].min()
    X_train.loc[idx, 'max_first_50000'] = xc[:50000].max()
    X_train.loc[idx, 'max_last_50000'] = xc[-50000:].max()
    X_train.loc[idx, 'max_first_10000'] = xc[:10000].max()
    X_train.loc[idx, 'max_last_10000'] = xc[-10000:].max()
    
    X_train.loc[idx, 'max_to_min'] = xc.max() / np.abs(xc.min())
    X_train.loc[idx, 'max_to_min_diff'] = xc.max() - np.abs(xc.min())
    X_train.loc[idx, 'count_big'] = len(xc[np.abs(xc) > 500])
    X_train.loc[idx, 'sum'] = xc.sum()
    
    X_train.loc[idx, 'mean_change_rate_first_50000'] = np.mean(np.nonzero((np.diff(xc[:50000]) / xc[:50000][:-1]))[0])
    X_train.loc[idx, 'mean_change_rate_last_50000'] = np.mean(np.nonzero((np.diff(xc[-50000:]) / xc[-50000:][:-1]))[0])
    X_train.loc[idx, 'mean_change_rate_first_10000'] = np.mean(np.nonzero((np.diff(xc[:10000]) / xc[:10000][:-1]))[0])
    X_train.loc[idx, 'mean_change_rate_last_10000'] = np.mean(np.nonzero((np.diff(xc[-10000:]) / xc[-10000:][:-1]))[0])
    
    X_train.loc[idx, 'q95'] = np.quantile(xc, 0.95)
    X_train.loc[idx, 'q99'] = np.quantile(xc, 0.99)
    X_train.loc[idx, 'q05'] = np.quantile(xc, 0.05)
    X_train.loc[idx, 'q01'] = np.quantile(xc, 0.01)
    X_train.loc[idx, 'abs_q95'] = np.quantile(np.abs(xc), 0.95)
    X_train.loc[idx, 'abs_q99'] = np.quantile(np.abs(xc), 0.99)
    X_train.loc[idx, 'abs_q05'] = np.quantile(np.abs(xc), 0.05)
    X_train.loc[idx, 'abs_q01'] = np.quantile(np.abs(xc), 0.01)
    
    X_train.loc[idx, 'trend'] = add_trend_feature(xc)
    X_train.loc[idx, 'abs_trend'] = add_trend_feature(xc, abs_values=True)
    X_train.loc[idx, 'abs_mean'] = np.abs(xc).mean()
    X_train.loc[idx, 'abs_std'] = np.abs(xc).std()

    X_train.loc[idx, 'mad'] = xc.mad()
    X_train.loc[idx, 'kurt'] = xc.kurtosis()
    X_train.loc[idx, 'skew'] = xc.skew()
    X_train.loc[idx, 'median'] = xc.median()
    
    X_train.loc[idx, 'Hilbert_mean'] = np.abs(hilbert(xc)).mean()
    X_train.loc[idx, 'Hann_window_mean_50'] = (convolve(xc, hann(50), mode='same')/sum(hann(50))).mean()
    X_train.loc[idx, 'Hann_window_mean_150'] = (convolve(xc, hann(150), mode='same')/sum(hann(150))).mean()
    X_train.loc[idx, 'Hann_window_mean_1500'] = (convolve(xc, hann(1500), mode='same')/sum(hann(1500))).mean()
    X_train.loc[idx, 'Hann_window_mean_15000'] = (convolve(xc, hann(15000), mode='same')/sum(hann(15000))).mean()
    X_train.loc[idx, 'classic_sta_lta1_mean'] = classic_sta_lta(xc, 500, 10000).mean()
    X_train.loc[idx, 'classic_sta_lta2_mean'] = classic_sta_lta(xc, 5000, 100000).mean()
    X_train.loc[idx, 'classic_sta_lta3_mean'] = classic_sta_lta(xc, 3333, 6666).mean()
    X_train.loc[idx, 'classic_sta_lta4_mean'] = classic_sta_lta(xc, 10000, 25000).mean()
    X_train.loc[idx, 'classic_sta_lta5_mean'] = classic_sta_lta(xc, 50, 1000).mean()
    X_train.loc[idx, 'classic_sta_lta6_mean'] = classic_sta_lta(xc, 100, 5000).mean()
    X_train.loc[idx, 'classic_sta_lta7_mean'] = classic_sta_lta(xc, 333, 666).mean()
    X_train.loc[idx, 'classic_sta_lta8_mean'] = classic_sta_lta(xc, 4000, 10000).mean()
    
    X_train.loc[idx, 'Moving_average_700_mean'] = xc.rolling(window=700).mean().mean(skipna=True)
    X_train.loc[idx, 'Moving_average_1500_mean'] = xc.rolling(window=1500).mean().mean(skipna=True)
    X_train.loc[idx, 'Moving_average_3000_mean'] = xc.rolling(window=3000).mean().mean(skipna=True)
    X_train.loc[idx, 'Moving_average_6000_mean'] = xc.rolling(window=6000).mean().mean(skipna=True)
    ewma = pd.Series.ewm
    X_train.loc[idx, 'exp_moving_average_300_mean'] = (ewma(xc, span=300).mean()).mean(skipna=True)
    X_train.loc[idx, 'exp_moving_average_3000_mean'] = (ewma(xc, span=3000).mean()).mean(skipna=True)
    X_train.loc[idx, 'exp_moving_average_30000_mean'] = (ewma(xc, span=30000).mean()).mean(skipna=True)
    X_train.loc[idx, 'exp_moving_average_50000_mean'] = (ewma(xc, span=50000).mean()).mean(skipna=True)
    
    X_train.loc[idx, 'MA_700MA_std_mean'] = xc.rolling(window=700).std().mean()
    X_train.loc[idx,'MA_700MA_BB_high_mean'] = (X_train.loc[idx, 'Moving_average_700_mean'] + 2 * X_train.loc[idx, 'MA_700MA_std_mean']).mean()
    X_train.loc[idx,'MA_700MA_BB_low_mean'] = (X_train.loc[idx, 'Moving_average_700_mean'] - 2 * X_train.loc[idx, 'MA_700MA_std_mean']).mean()
    X_train.loc[idx, 'MA_400MA_std_mean'] = xc.rolling(window=400).std().mean()
    X_train.loc[idx,'MA_400MA_BB_high_mean'] = (X_train.loc[idx, 'Moving_average_700_mean'] + 2 * X_train.loc[idx, 'MA_400MA_std_mean']).mean()
    X_train.loc[idx,'MA_400MA_BB_low_mean'] = (X_train.loc[idx, 'Moving_average_700_mean'] - 2 * X_train.loc[idx, 'MA_400MA_std_mean']).mean()
    X_train.loc[idx, 'MA_1000MA_std_mean'] = xc.rolling(window=1000).std().mean()
    
    X_train.loc[idx, 'iqr'] = np.subtract(*np.percentile(xc, [25, 75]))
    X_train.loc[idx, 'q999'] = np.quantile(xc, 0.999)
    X_train.loc[idx, 'q001'] = np.quantile(xc, 0.001)
    X_train.loc[idx, 'ave10'] = stats.trim_mean(xc, 0.1)
    
    X_train.loc[idx, 'number_peaks_50p'] = feature_calculators.number_peaks(xc.values, 50)
    X_train.loc[idx, 'number_peaks_100p'] = feature_calculators.number_peaks(xc.values, 100)
    X_train.loc[idx, 'number_peaks_500p'] = feature_calculators.number_peaks(xc.values, 500)
    X_train.loc[idx, 'number_peaks_1000p'] = feature_calculators.number_peaks(xc.values, 1000)
    X_train.loc[idx, 'number_peaks_10000p'] = feature_calculators.number_peaks(xc.values, 10000)
    X_train.loc[idx, 'autocorrelaion_10'] = feature_calculators.autocorrelation(xc.values, 10)
    X_train.loc[idx, 'autocorrelaion_50'] = feature_calculators.autocorrelation(xc.values, 50)
    X_train.loc[idx, 'autocorrelaion_100'] = feature_calculators.autocorrelation(xc.values, 100)
    X_train.loc[idx, 'autocorrelaion_1000'] = feature_calculators.autocorrelation(xc.values, 1000)
    X_train.loc[idx, 'c3_5'] = feature_calculators.c3(xc.values, 5)
    X_train.loc[idx, 'c3_10'] = feature_calculators.c3(xc.values, 10)
    X_train.loc[idx, 'c3_100'] = feature_calculators.c3(xc.values, 100)
    X_train.loc[idx, 'binned_entropy_50'] = feature_calculators.binned_entropy(xc.values, 50)
    X_train.loc[idx, 'binned_entropy_80'] = feature_calculators.binned_entropy(xc.values, 80)
    X_train.loc[idx, 'binned_entropy_100'] = feature_calculators.binned_entropy(xc.values, 100)
    X_train.loc[idx, 'binned_entropy_500'] = feature_calculators.binned_entropy(xc.values, 500)
    X_train.loc[idx, 'mean_abs_change'] = feature_calculators.mean_abs_change(xc.values)
 
    # FFT transform values
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    X_train.loc[idx, 'Rmean'] = realFFT.mean()
    X_train.loc[idx, 'Rstd'] = realFFT.std()
    X_train.loc[idx, 'Rmax'] = realFFT.max()
    X_train.loc[idx, 'Rmin'] = realFFT.min()
    X_train.loc[idx, 'Imean'] = imagFFT.mean()
    X_train.loc[idx, 'Istd'] = imagFFT.std()
    X_train.loc[idx, 'Imax'] = imagFFT.max()
    X_train.loc[idx, 'Imin'] = imagFFT.min()
    
    X_train.loc[idx, 'Rmean_last_5000'] = realFFT[-5000:].mean()
    X_train.loc[idx, 'Rstd_last_5000'] = realFFT[-5000:].std()
    X_train.loc[idx, 'Rmax_last_5000'] = realFFT[-5000:].max()
    X_train.loc[idx, 'Rmin_last_5000'] = realFFT[-5000:].min()
    X_train.loc[idx, 'Rmean_last_15000'] = realFFT[-15000:].mean()
    X_train.loc[idx, 'Rstd_last_15000'] = realFFT[-15000:].std()
    X_train.loc[idx, 'Rmax_last_15000'] = realFFT[-15000:].max()
    X_train.loc[idx, 'Rmin_last_15000'] = realFFT[-15000:].min() 
    
    for windows in [10, 50, 100, 500, 1000, 5000, 10000]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values

        X_train.loc[idx, f'mean_roll_{windows}_mean'] = x_roll_mean.mean()
        X_train.loc[idx, f'mean_roll_{windows}_std'] = x_roll_mean.std()
        X_train.loc[idx, f'mean_roll_{windows}_max'] = x_roll_mean.max()
        X_train.loc[idx, f'mean_roll_{windows}_min'] = x_roll_mean.min()
        X_train.loc[idx, f'mean_roll_{windows}_q01'] = np.quantile(x_roll_mean, 0.01)
        X_train.loc[idx, f'mean_roll_{windows}_q05'] = np.quantile(x_roll_mean, 0.05)
        X_train.loc[idx, f'mean_roll_{windows}_median'] = np.median(x_roll_mean)
        X_train.loc[idx, f'mean_roll_{windows}_q95'] = np.quantile(x_roll_mean, 0.95)
        X_train.loc[idx, f'mean_roll_{windows}_q99'] = np.quantile(x_roll_mean, 0.99)
        X_train.loc[idx, f'mean_roll_{windows}_av_change'] = np.mean(np.diff(x_roll_mean))
        
        X_train.loc[idx, f'std_roll_{windows}_mean'] = x_roll_std.mean()
        X_train.loc[idx, f'std_roll_{windows}_std'] = x_roll_std.std()
        X_train.loc[idx, f'std_roll_{windows}_max'] = x_roll_std.max()
        X_train.loc[idx, f'std_roll_{windows}_min'] = x_roll_std.min()
        X_train.loc[idx, f'std_roll_{windows}_q01'] = np.quantile(x_roll_std, 0.01)
        X_train.loc[idx, f'std_roll_{windows}_q05'] = np.quantile(x_roll_std, 0.05)
        X_train.loc[idx, f'std_roll_{windows}_median'] = np.median(x_roll_std)
        X_train.loc[idx, f'std_roll_{windows}_q95'] = np.quantile(x_roll_std, 0.95)
        X_train.loc[idx, f'std_roll_{windows}_q99'] = np.quantile(x_roll_std, 0.99)
        X_train.loc[idx, f'std_roll_{windows}_av_change'] = np.mean(np.diff(x_roll_std))       
    
    # ==================================================================
    # =====================denosing data================================
    # ==================================================================

    x = denoise_signal(high_pass_filter(segment.acoustic_data, low_cutoff=100000), 
                       wavelet='haar', level=1)
    xc = pd.DataFrame(x, columns=['signals']).signals
    zc = np.fft.fft(xc)
    
    X_train.loc[idx, 'den_mean'] = xc.mean()
    X_train.loc[idx, 'den_std'] =xc.std()
    X_train.loc[idx, 'den_max'] = xc.max()
    X_train.loc[idx, 'den_min'] = xc.min()   
    X_train.loc[idx, 'den_mean_change_abs'] = np.mean(np.diff(xc))
    X_train.loc[idx, 'den_mean_change_rate'] = np.mean(np.nonzero((np.diff(xc) / xc[:-1]))[0])
    X_train.loc[idx, 'den_abs_max'] = np.abs(xc).max()
    
    X_train.loc[idx, 'den_mean_first_50000'] = xc[:50000].mean()
    X_train.loc[idx, 'den_mean_last_50000'] = xc[-50000:].mean()
    X_train.loc[idx, 'den_mean_first_10000'] = xc[:10000].mean()
    X_train.loc[idx, 'den_mean_last_10000'] = xc[-10000:].mean()
    X_train.loc[idx, 'den_std_first_50000'] = xc[:50000].std()
    X_train.loc[idx, 'den_std_last_50000'] = xc[-50000:].std()
    X_train.loc[idx, 'den_std_first_10000'] = xc[:10000].std()
    X_train.loc[idx, 'den_std_last_10000'] = xc[-10000:].std()
    X_train.loc[idx, 'den_min_first_50000'] = xc[:50000].min()
    X_train.loc[idx, 'den_min_last_50000'] = xc[-50000:].min()
    X_train.loc[idx, 'den_min_first_10000'] = xc[:10000].min()
    X_train.loc[idx, 'den_min_last_10000'] = xc[-10000:].min()
    X_train.loc[idx, 'den_max_first_50000'] = xc[:50000].max()
    X_train.loc[idx, 'den_max_last_50000'] = xc[-50000:].max()
    X_train.loc[idx, 'den_max_first_10000'] = xc[:10000].max()
    X_train.loc[idx, 'den_max_last_10000'] = xc[-10000:].max()
    
    X_train.loc[idx, 'den_max_to_min'] = xc.max() / np.abs(xc.min())
    X_train.loc[idx, 'den_max_to_min_diff'] = xc.max() - np.abs(xc.min())
    X_train.loc[idx, 'den_count_big'] = len(xc[np.abs(xc) > 500])
    X_train.loc[idx, 'den_sum'] = xc.sum()
    
    X_train.loc[idx, 'den_mean_change_rate_first_50000'] = np.mean(np.nonzero((np.diff(xc[:50000]) / xc[:50000][:-1]))[0])
    X_train.loc[idx, 'den_mean_change_rate_last_50000'] = np.mean(np.nonzero((np.diff(xc[-50000:]) / xc[-50000:][:-1]))[0])
    X_train.loc[idx, 'den_mean_change_rate_first_10000'] = np.mean(np.nonzero((np.diff(xc[:10000]) / xc[:10000][:-1]))[0])
    X_train.loc[idx, 'den_mean_change_rate_last_10000'] = np.mean(np.nonzero((np.diff(xc[-10000:]) / xc[-10000:][:-1]))[0])
    
    X_train.loc[idx, 'den_q95'] = np.quantile(xc, 0.95)
    X_train.loc[idx, 'den_q99'] = np.quantile(xc, 0.99)
    X_train.loc[idx, 'den_q05'] = np.quantile(xc, 0.05)
    X_train.loc[idx, 'den_q01'] = np.quantile(xc, 0.01)
    X_train.loc[idx, 'den_abs_q95'] = np.quantile(np.abs(xc), 0.95)
    X_train.loc[idx, 'den_abs_q99'] = np.quantile(np.abs(xc), 0.99)
    X_train.loc[idx, 'den_abs_q05'] = np.quantile(np.abs(xc), 0.05)
    X_train.loc[idx, 'den_abs_q01'] = np.quantile(np.abs(xc), 0.01)
    
    X_train.loc[idx, 'den_trend'] = add_trend_feature(xc)
    X_train.loc[idx, 'den_abs_trend'] = add_trend_feature(xc, abs_values=True)
    X_train.loc[idx, 'den_abs_mean'] = np.abs(xc).mean()
    X_train.loc[idx, 'den_abs_std'] = np.abs(xc).std()

    X_train.loc[idx, 'den_mad'] = xc.mad()
    X_train.loc[idx, 'den_kurt'] = xc.kurtosis()
    X_train.loc[idx, 'den_skew'] = xc.skew()
    X_train.loc[idx, 'den_median'] = xc.median()
    
    X_train.loc[idx, 'den_Hilbert_mean'] = np.abs(hilbert(xc)).mean()
    X_train.loc[idx, 'den_Hann_window_mean_50'] = (convolve(xc, hann(50), mode='same')/sum(hann(50))).mean()
    X_train.loc[idx, 'den_Hann_window_mean_150'] = (convolve(xc, hann(150), mode='same')/sum(hann(150))).mean()
    X_train.loc[idx, 'den_Hann_window_mean_1500'] = (convolve(xc, hann(1500), mode='same')/sum(hann(1500))).mean()
    X_train.loc[idx, 'den_Hann_window_mean_15000'] = (convolve(xc, hann(15000), mode='same')/sum(hann(15000))).mean()
    X_train.loc[idx, 'den_classic_sta_lta1_mean'] = classic_sta_lta(xc, 500, 10000).mean()
    X_train.loc[idx, 'den_classic_sta_lta2_mean'] = classic_sta_lta(xc, 5000, 100000).mean()
    X_train.loc[idx, 'den_classic_sta_lta3_mean'] = classic_sta_lta(xc, 3333, 6666).mean()
    X_train.loc[idx, 'den_classic_sta_lta4_mean'] = classic_sta_lta(xc, 10000, 25000).mean()
    X_train.loc[idx, 'den_classic_sta_lta5_mean'] = classic_sta_lta(xc, 50, 1000).mean()
    X_train.loc[idx, 'den_classic_sta_lta6_mean'] = classic_sta_lta(xc, 100, 5000).mean()
    X_train.loc[idx, 'den_classic_sta_lta7_mean'] = classic_sta_lta(xc, 333, 666).mean()
    X_train.loc[idx, 'den_classic_sta_lta8_mean'] = classic_sta_lta(xc, 4000, 10000).mean()
    
    X_train.loc[idx, 'den_Moving_average_700_mean'] = xc.rolling(window=700).mean().mean(skipna=True)
    X_train.loc[idx, 'den_Moving_average_1500_mean'] = xc.rolling(window=1500).mean().mean(skipna=True)
    X_train.loc[idx, 'den_Moving_average_3000_mean'] = xc.rolling(window=3000).mean().mean(skipna=True)
    X_train.loc[idx, 'den_Moving_average_6000_mean'] = xc.rolling(window=6000).mean().mean(skipna=True)
    ewma = pd.Series.ewm
    X_train.loc[idx, 'den_exp_moving_average_300_mean'] = (ewma(xc, span=300).mean()).mean(skipna=True)
    X_train.loc[idx, 'den_exp_moving_average_3000_mean'] = (ewma(xc, span=3000).mean()).mean(skipna=True)
    X_train.loc[idx, 'den_exp_moving_average_30000_mean'] = (ewma(xc, span=30000).mean()).mean(skipna=True)
    X_train.loc[idx, 'den_exp_moving_average_50000_mean'] = (ewma(xc, span=50000).mean()).mean(skipna=True)
    
    X_train.loc[idx, 'den_MA_700MA_std_mean'] = xc.rolling(window=700).std().mean()
    X_train.loc[idx, 'den_MA_700MA_BB_high_mean'] = (X_train.loc[idx, 'den_Moving_average_700_mean'] + 2 * X_train.loc[idx, 'den_MA_700MA_std_mean']).mean()
    X_train.loc[idx, 'den_MA_700MA_BB_low_mean'] = (X_train.loc[idx, 'den_Moving_average_700_mean'] - 2 * X_train.loc[idx, 'den_MA_700MA_std_mean']).mean()
    X_train.loc[idx, 'den_MA_400MA_std_mean'] = xc.rolling(window=400).std().mean()
    X_train.loc[idx, 'den_MA_400MA_BB_high_mean'] = (X_train.loc[idx, 'den_Moving_average_700_mean'] + 2 * X_train.loc[idx, 'den_MA_400MA_std_mean']).mean()
    X_train.loc[idx, 'den_MA_400MA_BB_low_mean'] = (X_train.loc[idx, 'den_Moving_average_700_mean'] - 2 * X_train.loc[idx, 'den_MA_400MA_std_mean']).mean()
    X_train.loc[idx, 'den_MA_1000MA_std_mean'] = xc.rolling(window=1000).std().mean()
    
    X_train.loc[idx, 'den_iqr'] = np.subtract(*np.percentile(xc, [25, 75]))
    X_train.loc[idx, 'den_q999'] = np.quantile(xc, 0.999)
    X_train.loc[idx, 'den_q001'] = np.quantile(xc, 0.001)
    X_train.loc[idx, 'den_ave10'] = stats.trim_mean(xc, 0.1)
    
    X_train.loc[idx, 'den_number_peaks_50p'] = feature_calculators.number_peaks(xc.values, 50)
    X_train.loc[idx, 'den_number_peaks_100p'] = feature_calculators.number_peaks(xc.values, 100)
    X_train.loc[idx, 'den_number_peaks_500p'] = feature_calculators.number_peaks(xc.values, 500)
    X_train.loc[idx, 'den_number_peaks_1000p'] = feature_calculators.number_peaks(xc.values, 1000)
    X_train.loc[idx, 'den_number_peaks_10000p'] = feature_calculators.number_peaks(xc.values, 10000)
    X_train.loc[idx, 'den_autocorrelaion_10'] = feature_calculators.autocorrelation(xc.values, 10)
    X_train.loc[idx, 'den_autocorrelaion_50'] = feature_calculators.autocorrelation(xc.values, 50)
    X_train.loc[idx, 'den_autocorrelaion_100'] = feature_calculators.autocorrelation(xc.values, 100)
    X_train.loc[idx, 'den_autocorrelaion_1000'] = feature_calculators.autocorrelation(xc.values, 1000)
    X_train.loc[idx, 'den_c3_5'] = feature_calculators.c3(xc.values, 5)
    X_train.loc[idx, 'den_c3_10'] = feature_calculators.c3(xc.values, 10)
    X_train.loc[idx, 'den_c3_100'] = feature_calculators.c3(xc.values, 100)
    X_train.loc[idx, 'den_binned_entropy_50'] = feature_calculators.binned_entropy(xc.values, 50)
    X_train.loc[idx, 'den_binned_entropy_80'] = feature_calculators.binned_entropy(xc.values, 80)
    X_train.loc[idx, 'den_binned_entropy_100'] = feature_calculators.binned_entropy(xc.values, 100)
    X_train.loc[idx, 'den_binned_entropy_500'] = feature_calculators.binned_entropy(xc.values, 500)
    X_train.loc[idx, 'den_mean_abs_change'] = feature_calculators.mean_abs_change(xc.values)
 
    # FFT transform values
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    X_train.loc[idx, 'den_Rmean'] = realFFT.mean()
    X_train.loc[idx, 'den_Rstd'] = realFFT.std()
    X_train.loc[idx, 'den_Rmax'] = realFFT.max()
    X_train.loc[idx, 'den_Rmin'] = realFFT.min()
    X_train.loc[idx, 'den_Imean'] = imagFFT.mean()
    X_train.loc[idx, 'den_Istd'] = imagFFT.std()
    X_train.loc[idx, 'den_Imax'] = imagFFT.max()
    X_train.loc[idx, 'den_Imin'] = imagFFT.min()
    
    X_train.loc[idx, 'den_Rmean_last_5000'] = realFFT[-5000:].mean()
    X_train.loc[idx, 'den_Rstd_last_5000'] = realFFT[-5000:].std()
    X_train.loc[idx, 'den_Rmax_last_5000'] = realFFT[-5000:].max()
    X_train.loc[idx, 'den_Rmin_last_5000'] = realFFT[-5000:].min()
    X_train.loc[idx, 'den_Rmean_last_15000'] = realFFT[-15000:].mean()
    X_train.loc[idx, 'den_Rstd_last_15000'] = realFFT[-15000:].std()
    X_train.loc[idx, 'den_Rmax_last_15000'] = realFFT[-15000:].max()
    X_train.loc[idx, 'den_Rmin_last_15000'] = realFFT[-15000:].min() 
    
    for windows in [10, 50, 100, 500, 1000, 5000, 10000]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values

        X_train.loc[idx, f'den_mean_roll_{windows}_mean'] = x_roll_mean.mean()
        X_train.loc[idx, f'den_mean_roll_{windows}_std'] = x_roll_mean.std()
        X_train.loc[idx, f'den_mean_roll_{windows}_max'] = x_roll_mean.max()
        X_train.loc[idx, f'den_mean_roll_{windows}_min'] = x_roll_mean.min()
        X_train.loc[idx, f'den_mean_roll_{windows}_q01'] = np.quantile(x_roll_mean, 0.01)
        X_train.loc[idx, f'den_mean_roll_{windows}_q05'] = np.quantile(x_roll_mean, 0.05)
        X_train.loc[idx, f'den_mean_roll_{windows}_median'] = np.median(x_roll_mean)
        X_train.loc[idx, f'den_mean_roll_{windows}_q95'] = np.quantile(x_roll_mean, 0.95)
        X_train.loc[idx, f'den_mean_roll_{windows}_q99'] = np.quantile(x_roll_mean, 0.99)
        X_train.loc[idx, f'den_mean_roll_{windows}_av_change'] = np.mean(np.diff(x_roll_mean))
        
        X_train.loc[idx, f'den_std_roll_{windows}_mean'] = x_roll_std.mean()
        X_train.loc[idx, f'den_std_roll_{windows}_std'] = x_roll_std.std()
        X_train.loc[idx, f'den_std_roll_{windows}_max'] = x_roll_std.max()
        X_train.loc[idx, f'den_std_roll_{windows}_min'] = x_roll_std.min()
        X_train.loc[idx, f'den_std_roll_{windows}_q01'] = np.quantile(x_roll_std, 0.01)
        X_train.loc[idx, f'den_std_roll_{windows}_q05'] = np.quantile(x_roll_std, 0.05)
        X_train.loc[idx, f'den_std_roll_{windows}_median'] = np.median(x_roll_std)
        X_train.loc[idx, f'den_std_roll_{windows}_q95'] = np.quantile(x_roll_std, 0.95)
        X_train.loc[idx, f'den_std_roll_{windows}_q99'] = np.quantile(x_roll_std, 0.99)
        X_train.loc[idx, f'den_std_roll_{windows}_av_change'] = np.mean(np.diff(x_roll_std))       
        
    idx += 1

100%|██████████████| 2097/2097 [13:57:36<00:00, 23.25s/it]

ValueError: Invalid number of FFT data points (0) specified.

In [13]:
X_train.to_csv('./new_features/X_train_all_features2.csv', index=True, header=True)

In [12]:
X_train.loc[6291]

mean                                  4.527200000000000
std                                   4.015886080174593
max                                  94.000000000000000
min                                 -97.000000000000000
mean_change_abs                      -0.000040000266668
mean_change_rate                  74898.794000015230267
abs_max                              97.000000000000000
mean_first_50000                      4.398320000000000
mean_last_50000                       4.557200000000000
mean_first_10000                      4.473800000000000
mean_last_10000                       4.367000000000000
std_first_50000                       4.899867483067990
std_last_50000                        3.254612113601281
std_first_10000                       3.859015776111317
std_last_10000                        3.578965199494111
min_first_50000                     -97.000000000000000
min_last_50000                      -26.000000000000000
min_first_10000                     -26.00000000

In [6]:
submission = pd.read_csv('D:/kaggle/earthquake/sample_submission.csv', index_col='seg_id')
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [8]:
for idx in tqdm(list(submission.index.values)):
    
    file = 'D:/kaggle/earthquake/test/' + idx + '.csv'
    seg = pd.read_csv(file)
    
    # ==================================================================
    # =====================original data================================
    # ==================================================================
    
    xc = pd.Series(seg.acoustic_data.values)
    zc = np.fft.fft(xc)
    
    X_test.loc[idx, 'mean'] = xc.mean()
    X_test.loc[idx, 'std'] =xc.std()
    X_test.loc[idx, 'max'] = xc.max()
    X_test.loc[idx, 'min'] = xc.min()   
    X_test.loc[idx, 'mean_change_abs'] = np.mean(np.diff(xc))
    X_test.loc[idx, 'mean_change_rate'] = np.mean(np.nonzero((np.diff(xc) / xc[:-1]))[0])
    X_test.loc[idx, 'abs_max'] = np.abs(xc).max()
    
    X_test.loc[idx, 'mean_first_50000'] = xc[:50000].mean()
    X_test.loc[idx, 'mean_last_50000'] = xc[-50000:].mean()
    X_test.loc[idx, 'mean_first_10000'] = xc[:10000].mean()
    X_test.loc[idx, 'mean_last_10000'] = xc[-10000:].mean()
    X_test.loc[idx, 'std_first_50000'] = xc[:50000].std()
    X_test.loc[idx, 'std_last_50000'] = xc[-50000:].std()
    X_test.loc[idx, 'std_first_10000'] = xc[:10000].std()
    X_test.loc[idx, 'std_last_10000'] = xc[-10000:].std()
    X_test.loc[idx, 'min_first_50000'] = xc[:50000].min()
    X_test.loc[idx, 'min_last_50000'] = xc[-50000:].min()
    X_test.loc[idx, 'min_first_10000'] = xc[:10000].min()
    X_test.loc[idx, 'min_last_10000'] = xc[-10000:].min()
    X_test.loc[idx, 'max_first_50000'] = xc[:50000].max()
    X_test.loc[idx, 'max_last_50000'] = xc[-50000:].max()
    X_test.loc[idx, 'max_first_10000'] = xc[:10000].max()
    X_test.loc[idx, 'max_last_10000'] = xc[-10000:].max()
    
    X_test.loc[idx, 'max_to_min'] = xc.max() / np.abs(xc.min())
    X_test.loc[idx, 'max_to_min_diff'] = xc.max() - np.abs(xc.min())
    X_test.loc[idx, 'count_big'] = len(xc[np.abs(xc) > 500])
    X_test.loc[idx, 'sum'] = xc.sum()
    
    X_test.loc[idx, 'mean_change_rate_first_50000'] = np.mean(np.nonzero((np.diff(xc[:50000]) / xc[:50000][:-1]))[0])
    X_test.loc[idx, 'mean_change_rate_last_50000'] = np.mean(np.nonzero((np.diff(xc[-50000:]) / xc[-50000:][:-1]))[0])
    X_test.loc[idx, 'mean_change_rate_first_10000'] = np.mean(np.nonzero((np.diff(xc[:10000]) / xc[:10000][:-1]))[0])
    X_test.loc[idx, 'mean_change_rate_last_10000'] = np.mean(np.nonzero((np.diff(xc[-10000:]) / xc[-10000:][:-1]))[0])
    
    X_test.loc[idx, 'q95'] = np.quantile(xc, 0.95)
    X_test.loc[idx, 'q99'] = np.quantile(xc, 0.99)
    X_test.loc[idx, 'q05'] = np.quantile(xc, 0.05)
    X_test.loc[idx, 'q01'] = np.quantile(xc, 0.01)
    X_test.loc[idx, 'abs_q95'] = np.quantile(np.abs(xc), 0.95)
    X_test.loc[idx, 'abs_q99'] = np.quantile(np.abs(xc), 0.99)
    X_test.loc[idx, 'abs_q05'] = np.quantile(np.abs(xc), 0.05)
    X_test.loc[idx, 'abs_q01'] = np.quantile(np.abs(xc), 0.01)
    
    X_test.loc[idx, 'trend'] = add_trend_feature(xc)
    X_test.loc[idx, 'abs_trend'] = add_trend_feature(xc, abs_values=True)
    X_test.loc[idx, 'abs_mean'] = np.abs(xc).mean()
    X_test.loc[idx, 'abs_std'] = np.abs(xc).std()

    X_test.loc[idx, 'mad'] = xc.mad()
    X_test.loc[idx, 'kurt'] = xc.kurtosis()
    X_test.loc[idx, 'skew'] = xc.skew()
    X_test.loc[idx, 'median'] = xc.median()
    
    X_test.loc[idx, 'Hilbert_mean'] = np.abs(hilbert(xc)).mean()
    X_test.loc[idx, 'Hann_window_mean_50'] = (convolve(xc, hann(50), mode='same')/sum(hann(50))).mean()
    X_test.loc[idx, 'Hann_window_mean_150'] = (convolve(xc, hann(150), mode='same')/sum(hann(150))).mean()
    X_test.loc[idx, 'Hann_window_mean_1500'] = (convolve(xc, hann(1500), mode='same')/sum(hann(1500))).mean()
    X_test.loc[idx, 'Hann_window_mean_15000'] = (convolve(xc, hann(15000), mode='same')/sum(hann(15000))).mean()
    X_test.loc[idx, 'classic_sta_lta1_mean'] = classic_sta_lta(xc, 500, 10000).mean()
    X_test.loc[idx, 'classic_sta_lta2_mean'] = classic_sta_lta(xc, 5000, 100000).mean()
    X_test.loc[idx, 'classic_sta_lta3_mean'] = classic_sta_lta(xc, 3333, 6666).mean()
    X_test.loc[idx, 'classic_sta_lta4_mean'] = classic_sta_lta(xc, 10000, 25000).mean()
    X_test.loc[idx, 'classic_sta_lta5_mean'] = classic_sta_lta(xc, 50, 1000).mean()
    X_test.loc[idx, 'classic_sta_lta6_mean'] = classic_sta_lta(xc, 100, 5000).mean()
    X_test.loc[idx, 'classic_sta_lta7_mean'] = classic_sta_lta(xc, 333, 666).mean()
    X_test.loc[idx, 'classic_sta_lta8_mean'] = classic_sta_lta(xc, 4000, 10000).mean()
    
    X_test.loc[idx, 'Moving_average_700_mean'] = xc.rolling(window=700).mean().mean(skipna=True)
    X_test.loc[idx, 'Moving_average_1500_mean'] = xc.rolling(window=1500).mean().mean(skipna=True)
    X_test.loc[idx, 'Moving_average_3000_mean'] = xc.rolling(window=3000).mean().mean(skipna=True)
    X_test.loc[idx, 'Moving_average_6000_mean'] = xc.rolling(window=6000).mean().mean(skipna=True)
    ewma = pd.Series.ewm
    X_test.loc[idx, 'exp_moving_average_300_mean'] = (ewma(xc, span=300).mean()).mean(skipna=True)
    X_test.loc[idx, 'exp_moving_average_3000_mean'] = (ewma(xc, span=3000).mean()).mean(skipna=True)
    X_test.loc[idx, 'exp_moving_average_30000_mean'] = (ewma(xc, span=30000).mean()).mean(skipna=True)
    X_test.loc[idx, 'exp_moving_average_50000_mean'] = (ewma(xc, span=50000).mean()).mean(skipna=True)
    
    X_test.loc[idx, 'MA_700MA_std_mean'] = xc.rolling(window=700).std().mean()
    X_test.loc[idx,'MA_700MA_BB_high_mean'] = (X_test.loc[idx, 'Moving_average_700_mean'] + 2 * X_test.loc[idx, 'MA_700MA_std_mean']).mean()
    X_test.loc[idx,'MA_700MA_BB_low_mean'] = (X_test.loc[idx, 'Moving_average_700_mean'] - 2 * X_test.loc[idx, 'MA_700MA_std_mean']).mean()
    X_test.loc[idx, 'MA_400MA_std_mean'] = xc.rolling(window=400).std().mean()
    X_test.loc[idx,'MA_400MA_BB_high_mean'] = (X_test.loc[idx, 'Moving_average_700_mean'] + 2 * X_test.loc[idx, 'MA_400MA_std_mean']).mean()
    X_test.loc[idx,'MA_400MA_BB_low_mean'] = (X_test.loc[idx, 'Moving_average_700_mean'] - 2 * X_test.loc[idx, 'MA_400MA_std_mean']).mean()
    X_test.loc[idx, 'MA_1000MA_std_mean'] = xc.rolling(window=1000).std().mean()
    
    X_test.loc[idx, 'iqr'] = np.subtract(*np.percentile(xc, [25, 75]))
    X_test.loc[idx, 'q999'] = np.quantile(xc, 0.999)
    X_test.loc[idx, 'q001'] = np.quantile(xc, 0.001)
    X_test.loc[idx, 'ave10'] = stats.trim_mean(xc, 0.1)
    
    X_test.loc[idx, 'number_peaks_50p'] = feature_calculators.number_peaks(xc.values, 50)
    X_test.loc[idx, 'number_peaks_100p'] = feature_calculators.number_peaks(xc.values, 100)
    X_test.loc[idx, 'number_peaks_500p'] = feature_calculators.number_peaks(xc.values, 500)
    X_test.loc[idx, 'number_peaks_1000p'] = feature_calculators.number_peaks(xc.values, 1000)
    X_test.loc[idx, 'number_peaks_10000p'] = feature_calculators.number_peaks(xc.values, 10000)
    X_test.loc[idx, 'autocorrelaion_10'] = feature_calculators.autocorrelation(xc.values, 10)
    X_test.loc[idx, 'autocorrelaion_50'] = feature_calculators.autocorrelation(xc.values, 50)
    X_test.loc[idx, 'autocorrelaion_100'] = feature_calculators.autocorrelation(xc.values, 100)
    X_test.loc[idx, 'autocorrelaion_1000'] = feature_calculators.autocorrelation(xc.values, 1000)
    X_test.loc[idx, 'c3_5'] = feature_calculators.c3(xc.values, 5)
    X_test.loc[idx, 'c3_10'] = feature_calculators.c3(xc.values, 10)
    X_test.loc[idx, 'c3_100'] = feature_calculators.c3(xc.values, 100)
    X_test.loc[idx, 'binned_entropy_50'] = feature_calculators.binned_entropy(xc.values, 50)
    X_test.loc[idx, 'binned_entropy_80'] = feature_calculators.binned_entropy(xc.values, 80)
    X_test.loc[idx, 'binned_entropy_100'] = feature_calculators.binned_entropy(xc.values, 100)
    X_test.loc[idx, 'binned_entropy_500'] = feature_calculators.binned_entropy(xc.values, 500)
    X_test.loc[idx, 'mean_abs_change'] = feature_calculators.mean_abs_change(xc.values)
 
    # FFT transform values
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    X_test.loc[idx, 'Rmean'] = realFFT.mean()
    X_test.loc[idx, 'Rstd'] = realFFT.std()
    X_test.loc[idx, 'Rmax'] = realFFT.max()
    X_test.loc[idx, 'Rmin'] = realFFT.min()
    X_test.loc[idx, 'Imean'] = imagFFT.mean()
    X_test.loc[idx, 'Istd'] = imagFFT.std()
    X_test.loc[idx, 'Imax'] = imagFFT.max()
    X_test.loc[idx, 'Imin'] = imagFFT.min()
    
    X_test.loc[idx, 'Rmean_last_5000'] = realFFT[-5000:].mean()
    X_test.loc[idx, 'Rstd_last_5000'] = realFFT[-5000:].std()
    X_test.loc[idx, 'Rmax_last_5000'] = realFFT[-5000:].max()
    X_test.loc[idx, 'Rmin_last_5000'] = realFFT[-5000:].min()
    X_test.loc[idx, 'Rmean_last_15000'] = realFFT[-15000:].mean()
    X_test.loc[idx, 'Rstd_last_15000'] = realFFT[-15000:].std()
    X_test.loc[idx, 'Rmax_last_15000'] = realFFT[-15000:].max()
    X_test.loc[idx, 'Rmin_last_15000'] = realFFT[-15000:].min() 
    
    for windows in [10, 50, 100, 500, 1000, 5000, 10000]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values

        X_test.loc[idx, f'mean_roll_{windows}_mean'] = x_roll_mean.mean()
        X_test.loc[idx, f'mean_roll_{windows}_std'] = x_roll_mean.std()
        X_test.loc[idx, f'mean_roll_{windows}_max'] = x_roll_mean.max()
        X_test.loc[idx, f'mean_roll_{windows}_min'] = x_roll_mean.min()
        X_test.loc[idx, f'mean_roll_{windows}_q01'] = np.quantile(x_roll_mean, 0.01)
        X_test.loc[idx, f'mean_roll_{windows}_q05'] = np.quantile(x_roll_mean, 0.05)
        X_test.loc[idx, f'mean_roll_{windows}_median'] = np.median(x_roll_mean)
        X_test.loc[idx, f'mean_roll_{windows}_q95'] = np.quantile(x_roll_mean, 0.95)
        X_test.loc[idx, f'mean_roll_{windows}_q99'] = np.quantile(x_roll_mean, 0.99)
        X_test.loc[idx, f'mean_roll_{windows}_av_change'] = np.mean(np.diff(x_roll_mean))
        
        X_test.loc[idx, f'std_roll_{windows}_mean'] = x_roll_std.mean()
        X_test.loc[idx, f'std_roll_{windows}_std'] = x_roll_std.std()
        X_test.loc[idx, f'std_roll_{windows}_max'] = x_roll_std.max()
        X_test.loc[idx, f'std_roll_{windows}_min'] = x_roll_std.min()
        X_test.loc[idx, f'std_roll_{windows}_q01'] = np.quantile(x_roll_std, 0.01)
        X_test.loc[idx, f'std_roll_{windows}_q05'] = np.quantile(x_roll_std, 0.05)
        X_test.loc[idx, f'std_roll_{windows}_median'] = np.median(x_roll_std)
        X_test.loc[idx, f'std_roll_{windows}_q95'] = np.quantile(x_roll_std, 0.95)
        X_test.loc[idx, f'std_roll_{windows}_q99'] = np.quantile(x_roll_std, 0.99)
        X_test.loc[idx, f'std_roll_{windows}_av_change'] = np.mean(np.diff(x_roll_std))       
        
    
    # ==================================================================
    # =====================denosing data================================
    # ==================================================================
    
    x = denoise_signal(high_pass_filter(seg.acoustic_data, low_cutoff=100000), 
                       wavelet='haar', level=1)
    xc = pd.DataFrame(x, columns=['signals']).signals
    zc = np.fft.fft(xc)
    
    X_test.loc[idx, 'den_mean'] = xc.mean()
    X_test.loc[idx, 'den_std'] =xc.std()
    X_test.loc[idx, 'den_max'] = xc.max()
    X_test.loc[idx, 'den_min'] = xc.min()   
    X_test.loc[idx, 'den_mean_change_abs'] = np.mean(np.diff(xc))
    X_test.loc[idx, 'den_mean_change_rate'] = np.mean(np.nonzero((np.diff(xc) / xc[:-1]))[0])
    X_test.loc[idx, 'den_abs_max'] = np.abs(xc).max()
    
    X_test.loc[idx, 'den_mean_first_50000'] = xc[:50000].mean()
    X_test.loc[idx, 'den_mean_last_50000'] = xc[-50000:].mean()
    X_test.loc[idx, 'den_mean_first_10000'] = xc[:10000].mean()
    X_test.loc[idx, 'den_mean_last_10000'] = xc[-10000:].mean()
    X_test.loc[idx, 'den_std_first_50000'] = xc[:50000].std()
    X_test.loc[idx, 'den_std_last_50000'] = xc[-50000:].std()
    X_test.loc[idx, 'den_std_first_10000'] = xc[:10000].std()
    X_test.loc[idx, 'den_std_last_10000'] = xc[-10000:].std()
    X_test.loc[idx, 'den_min_first_50000'] = xc[:50000].min()
    X_test.loc[idx, 'den_min_last_50000'] = xc[-50000:].min()
    X_test.loc[idx, 'den_min_first_10000'] = xc[:10000].min()
    X_test.loc[idx, 'den_min_last_10000'] = xc[-10000:].min()
    X_test.loc[idx, 'den_max_first_50000'] = xc[:50000].max()
    X_test.loc[idx, 'den_max_last_50000'] = xc[-50000:].max()
    X_test.loc[idx, 'den_max_first_10000'] = xc[:10000].max()
    X_test.loc[idx, 'den_max_last_10000'] = xc[-10000:].max()
    
    X_test.loc[idx, 'den_max_to_min'] = xc.max() / np.abs(xc.min())
    X_test.loc[idx, 'den_max_to_min_diff'] = xc.max() - np.abs(xc.min())
    X_test.loc[idx, 'den_count_big'] = len(xc[np.abs(xc) > 500])
    X_test.loc[idx, 'den_sum'] = xc.sum()
    
    X_test.loc[idx, 'den_mean_change_rate_first_50000'] = np.mean(np.nonzero((np.diff(xc[:50000]) / xc[:50000][:-1]))[0])
    X_test.loc[idx, 'den_mean_change_rate_last_50000'] = np.mean(np.nonzero((np.diff(xc[-50000:]) / xc[-50000:][:-1]))[0])
    X_test.loc[idx, 'den_mean_change_rate_first_10000'] = np.mean(np.nonzero((np.diff(xc[:10000]) / xc[:10000][:-1]))[0])
    X_test.loc[idx, 'den_mean_change_rate_last_10000'] = np.mean(np.nonzero((np.diff(xc[-10000:]) / xc[-10000:][:-1]))[0])
    
    X_test.loc[idx, 'den_q95'] = np.quantile(xc, 0.95)
    X_test.loc[idx, 'den_q99'] = np.quantile(xc, 0.99)
    X_test.loc[idx, 'den_q05'] = np.quantile(xc, 0.05)
    X_test.loc[idx, 'den_q01'] = np.quantile(xc, 0.01)
    X_test.loc[idx, 'den_abs_q95'] = np.quantile(np.abs(xc), 0.95)
    X_test.loc[idx, 'den_abs_q99'] = np.quantile(np.abs(xc), 0.99)
    X_test.loc[idx, 'den_abs_q05'] = np.quantile(np.abs(xc), 0.05)
    X_test.loc[idx, 'den_abs_q01'] = np.quantile(np.abs(xc), 0.01)
    
    X_test.loc[idx, 'den_trend'] = add_trend_feature(xc)
    X_test.loc[idx, 'den_abs_trend'] = add_trend_feature(xc, abs_values=True)
    X_test.loc[idx, 'den_abs_mean'] = np.abs(xc).mean()
    X_test.loc[idx, 'den_abs_std'] = np.abs(xc).std()

    X_test.loc[idx, 'den_mad'] = xc.mad()
    X_test.loc[idx, 'den_kurt'] = xc.kurtosis()
    X_test.loc[idx, 'den_skew'] = xc.skew()
    X_test.loc[idx, 'den_median'] = xc.median()
    
    X_test.loc[idx, 'den_Hilbert_mean'] = np.abs(hilbert(xc)).mean()
    X_test.loc[idx, 'den_Hann_window_mean_50'] = (convolve(xc, hann(50), mode='same')/sum(hann(50))).mean()
    X_test.loc[idx, 'den_Hann_window_mean_150'] = (convolve(xc, hann(150), mode='same')/sum(hann(150))).mean()
    X_test.loc[idx, 'den_Hann_window_mean_1500'] = (convolve(xc, hann(1500), mode='same')/sum(hann(1500))).mean()
    X_test.loc[idx, 'den_Hann_window_mean_15000'] = (convolve(xc, hann(15000), mode='same')/sum(hann(15000))).mean()
    X_test.loc[idx, 'den_classic_sta_lta1_mean'] = classic_sta_lta(xc, 500, 10000).mean()
    X_test.loc[idx, 'den_classic_sta_lta2_mean'] = classic_sta_lta(xc, 5000, 100000).mean()
    X_test.loc[idx, 'den_classic_sta_lta3_mean'] = classic_sta_lta(xc, 3333, 6666).mean()
    X_test.loc[idx, 'den_classic_sta_lta4_mean'] = classic_sta_lta(xc, 10000, 25000).mean()
    X_test.loc[idx, 'den_classic_sta_lta5_mean'] = classic_sta_lta(xc, 50, 1000).mean()
    X_test.loc[idx, 'den_classic_sta_lta6_mean'] = classic_sta_lta(xc, 100, 5000).mean()
    X_test.loc[idx, 'den_classic_sta_lta7_mean'] = classic_sta_lta(xc, 333, 666).mean()
    X_test.loc[idx, 'den_classic_sta_lta8_mean'] = classic_sta_lta(xc, 4000, 10000).mean()
    
    X_test.loc[idx, 'den_Moving_average_700_mean'] = xc.rolling(window=700).mean().mean(skipna=True)
    X_test.loc[idx, 'den_Moving_average_1500_mean'] = xc.rolling(window=1500).mean().mean(skipna=True)
    X_test.loc[idx, 'den_Moving_average_3000_mean'] = xc.rolling(window=3000).mean().mean(skipna=True)
    X_test.loc[idx, 'den_Moving_average_6000_mean'] = xc.rolling(window=6000).mean().mean(skipna=True)
    ewma = pd.Series.ewm
    X_test.loc[idx, 'den_exp_moving_average_300_mean'] = (ewma(xc, span=300).mean()).mean(skipna=True)
    X_test.loc[idx, 'den_exp_moving_average_3000_mean'] = (ewma(xc, span=3000).mean()).mean(skipna=True)
    X_test.loc[idx, 'den_exp_moving_average_30000_mean'] = (ewma(xc, span=30000).mean()).mean(skipna=True)
    X_test.loc[idx, 'den_exp_moving_average_50000_mean'] = (ewma(xc, span=50000).mean()).mean(skipna=True)
    
    X_test.loc[idx, 'den_MA_700MA_std_mean'] = xc.rolling(window=700).std().mean()
    X_test.loc[idx, 'den_MA_700MA_BB_high_mean'] = (X_test.loc[idx, 'den_Moving_average_700_mean'] + 2 * X_test.loc[idx, 'den_MA_700MA_std_mean']).mean()
    X_test.loc[idx, 'den_MA_700MA_BB_low_mean'] = (X_test.loc[idx, 'den_Moving_average_700_mean'] - 2 * X_test.loc[idx, 'den_MA_700MA_std_mean']).mean()
    X_test.loc[idx, 'den_MA_400MA_std_mean'] = xc.rolling(window=400).std().mean()
    X_test.loc[idx, 'den_MA_400MA_BB_high_mean'] = (X_test.loc[idx, 'den_Moving_average_700_mean'] + 2 * X_test.loc[idx, 'den_MA_400MA_std_mean']).mean()
    X_test.loc[idx, 'den_MA_400MA_BB_low_mean'] = (X_test.loc[idx, 'den_Moving_average_700_mean'] - 2 * X_test.loc[idx, 'den_MA_400MA_std_mean']).mean()
    X_test.loc[idx, 'den_MA_1000MA_std_mean'] = xc.rolling(window=1000).std().mean()
    
    X_test.loc[idx, 'den_iqr'] = np.subtract(*np.percentile(xc, [25, 75]))
    X_test.loc[idx, 'den_q999'] = np.quantile(xc, 0.999)
    X_test.loc[idx, 'den_q001'] = np.quantile(xc, 0.001)
    X_test.loc[idx, 'den_ave10'] = stats.trim_mean(xc, 0.1)
    
    X_test.loc[idx, 'den_number_peaks_50p'] = feature_calculators.number_peaks(xc.values, 50)
    X_test.loc[idx, 'den_number_peaks_100p'] = feature_calculators.number_peaks(xc.values, 100)
    X_test.loc[idx, 'den_number_peaks_500p'] = feature_calculators.number_peaks(xc.values, 500)
    X_test.loc[idx, 'den_number_peaks_1000p'] = feature_calculators.number_peaks(xc.values, 1000)
    X_test.loc[idx, 'den_number_peaks_10000p'] = feature_calculators.number_peaks(xc.values, 10000)
    X_test.loc[idx, 'den_autocorrelaion_10'] = feature_calculators.autocorrelation(xc.values, 10)
    X_test.loc[idx, 'den_autocorrelaion_50'] = feature_calculators.autocorrelation(xc.values, 50)
    X_test.loc[idx, 'den_autocorrelaion_100'] = feature_calculators.autocorrelation(xc.values, 100)
    X_test.loc[idx, 'den_autocorrelaion_1000'] = feature_calculators.autocorrelation(xc.values, 1000)
    X_test.loc[idx, 'den_c3_5'] = feature_calculators.c3(xc.values, 5)
    X_test.loc[idx, 'den_c3_10'] = feature_calculators.c3(xc.values, 10)
    X_test.loc[idx, 'den_c3_100'] = feature_calculators.c3(xc.values, 100)
    X_test.loc[idx, 'den_binned_entropy_50'] = feature_calculators.binned_entropy(xc.values, 50)
    X_test.loc[idx, 'den_binned_entropy_80'] = feature_calculators.binned_entropy(xc.values, 80)
    X_test.loc[idx, 'den_binned_entropy_100'] = feature_calculators.binned_entropy(xc.values, 100)
    X_test.loc[idx, 'den_binned_entropy_500'] = feature_calculators.binned_entropy(xc.values, 500)
    X_test.loc[idx, 'den_mean_abs_change'] = feature_calculators.mean_abs_change(xc.values)
 
    # FFT transform values
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    X_test.loc[idx, 'den_Rmean'] = realFFT.mean()
    X_test.loc[idx, 'den_Rstd'] = realFFT.std()
    X_test.loc[idx, 'den_Rmax'] = realFFT.max()
    X_test.loc[idx, 'den_Rmin'] = realFFT.min()
    X_test.loc[idx, 'den_Imean'] = imagFFT.mean()
    X_test.loc[idx, 'den_Istd'] = imagFFT.std()
    X_test.loc[idx, 'den_Imax'] = imagFFT.max()
    X_test.loc[idx, 'den_Imin'] = imagFFT.min()
    
    X_test.loc[idx, 'den_Rmean_last_5000'] = realFFT[-5000:].mean()
    X_test.loc[idx, 'den_Rstd_last_5000'] = realFFT[-5000:].std()
    X_test.loc[idx, 'den_Rmax_last_5000'] = realFFT[-5000:].max()
    X_test.loc[idx, 'den_Rmin_last_5000'] = realFFT[-5000:].min()
    X_test.loc[idx, 'den_Rmean_last_15000'] = realFFT[-15000:].mean()
    X_test.loc[idx, 'den_Rstd_last_15000'] = realFFT[-15000:].std()
    X_test.loc[idx, 'den_Rmax_last_15000'] = realFFT[-15000:].max()
    X_test.loc[idx, 'den_Rmin_last_15000'] = realFFT[-15000:].min() 
    
    for windows in [10, 50, 100, 500, 1000, 5000, 10000]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values

        X_test.loc[idx, f'den_mean_roll_{windows}_mean'] = x_roll_mean.mean()
        X_test.loc[idx, f'den_mean_roll_{windows}_std'] = x_roll_mean.std()
        X_test.loc[idx, f'den_mean_roll_{windows}_max'] = x_roll_mean.max()
        X_test.loc[idx, f'den_mean_roll_{windows}_min'] = x_roll_mean.min()
        X_test.loc[idx, f'den_mean_roll_{windows}_q01'] = np.quantile(x_roll_mean, 0.01)
        X_test.loc[idx, f'den_mean_roll_{windows}_q05'] = np.quantile(x_roll_mean, 0.05)
        X_test.loc[idx, f'den_mean_roll_{windows}_median'] = np.median(x_roll_mean)
        X_test.loc[idx, f'den_mean_roll_{windows}_q95'] = np.quantile(x_roll_mean, 0.95)
        X_test.loc[idx, f'den_mean_roll_{windows}_q99'] = np.quantile(x_roll_mean, 0.99)
        X_test.loc[idx, f'den_mean_roll_{windows}_av_change'] = np.mean(np.diff(x_roll_mean))
        
        X_test.loc[idx, f'den_std_roll_{windows}_mean'] = x_roll_std.mean()
        X_test.loc[idx, f'den_std_roll_{windows}_std'] = x_roll_std.std()
        X_test.loc[idx, f'den_std_roll_{windows}_max'] = x_roll_std.max()
        X_test.loc[idx, f'den_std_roll_{windows}_min'] = x_roll_std.min()
        X_test.loc[idx, f'den_std_roll_{windows}_q01'] = np.quantile(x_roll_std, 0.01)
        X_test.loc[idx, f'den_std_roll_{windows}_q05'] = np.quantile(x_roll_std, 0.05)
        X_test.loc[idx, f'den_std_roll_{windows}_median'] = np.median(x_roll_std)
        X_test.loc[idx, f'den_std_roll_{windows}_q95'] = np.quantile(x_roll_std, 0.95)
        X_test.loc[idx, f'den_std_roll_{windows}_q99'] = np.quantile(x_roll_std, 0.99)
        X_test.loc[idx, f'den_std_roll_{windows}_av_change'] = np.mean(np.diff(x_roll_std))   
    

100%|██████████████| 2624/2624 [15:33:11<00:00, 20.73s/it]


In [10]:
X_test.to_csv('./new_features/X_test_all_features.csv', index=True, header=True)

In [12]:
X_test.head()

,mean,std,max,min,mean_change_abs,mean_change_rate,abs_max,mean_first_50000,mean_last_50000,mean_first_10000,...,den_std_roll_10000_mean,den_std_roll_10000_std,den_std_roll_10000_max,den_std_roll_10000_min,den_std_roll_10000_q01,den_std_roll_10000_q05,den_std_roll_10000_median,den_std_roll_10000_q95,den_std_roll_10000_q99,den_std_roll_10000_av_change
seg_id,,,,,,,,,,,,,,,,,,,,,
seg_00030f,4.491780000000000,4.893689687028083,115.0,-75.0,0.000026666844446,75044.983541275520111,115.0,4.46644,4.49598,4.3842,...,2.760738879234107,1.767032295683657,6.967250708302509,0.164405001196263,0.412846000585372,0.826215392499982,2.507467554558998,6.888663857287961,6.945255249711699,0.000005375311823
seg_0012b5,4.171153333333334,5.922839443206501,152.0,-140.0,-0.000013333422223,74949.685652956031845,152.0,4.01786,4.24444,4.0635,...,3.895347848312298,2.597804433881185,9.684650699455403,0.621692447357012,0.621692447357012,1.005311049555419,2.724245903661253,8.196541548667495,9.670105862920673,0.000002960198122
seg_00184e,4.610260000000000,6.946990077499337,248.0,-193.0,-0.000020000133334,74997.632054261513986,248.0,4.55518,4.55380,4.2452,...,4.161058344029043,4.262120691132202,18.718209207638580,0.505298887790425,0.505298887790425,0.666166329089699,2.495974326304224,15.550837465973933,18.710928539918228,-0.000011807302605
seg_003339,4.531473333333333,4.114146602958790,85.0,-93.0,0.000046666977780,74997.954642264914582,93.0,4.49052,4.48922,4.3834,...,2.188771447927612,1.763563346630023,7.156822811026635,0.000000091560300,0.000000357584224,0.000000357584224,1.819209140846672,7.085477479461632,7.156822811026635,-0.000016477429210
seg_0042cc,4.128340000000000,5.797163636219714,177.0,-147.0,-0.000006666711111,75075.108146992672118,177.0,4.23020,4.10524,4.4902,...,3.511429543748389,3.108405193585451,12.939748054498009,0.203877298025122,0.305003526711859,0.622230050386334,2.664548667326618,12.652994419268135,12.937472170513461,0.000012995710112


In [13]:
X_train.shape

(4195, 504)

In [ ]:
NUM_EVALS = 1000
N_FOLDS = 5
XGB_MAX_LEAVES = 2**12 
XGB_MAX_DEPTH = 50
EVAL_METRIC_XGB_REG = 'mae'

In [ ]:
def quick_hyperopt(data, labels, num_evals=NUM_EVALS, diagnostic=False):
        
    print('Running {} rounds of XGBoost parameter optimisation:'.format(num_evals))
    #clear space
    gc.collect()

    integer_params = ['max_depth']

    def objective(space_params):

        for param in integer_params:
            space_params[param] = int(space_params[param])

        #extract multiple nested tree_method conditional parameters
        #libera te tutemet ex inferis
        if space_params['tree_method']['tree_method'] == 'hist':
            max_bin = space_params['tree_method'].get('max_bin')
            space_params['max_bin'] = int(max_bin)
            if space_params['tree_method']['grow_policy']['grow_policy']['grow_policy'] == 'depthwise':
                grow_policy = space_params['tree_method'].get('grow_policy').get('grow_policy').get('grow_policy')
                space_params['grow_policy'] = grow_policy
                space_params['tree_method'] = 'hist'
            else:
                max_leaves = space_params['tree_method']['grow_policy']['grow_policy'].get('max_leaves')
                space_params['grow_policy'] = 'lossguide'
                space_params['max_leaves'] = int(max_leaves)
                space_params['tree_method'] = 'hist'
        else:
            space_params['tree_method'] = space_params['tree_method'].get('tree_method')

        #for classification replace EVAL_METRIC_XGB_REG with EVAL_METRIC_XGB_CLASS
        cv_results = xgb.cv(space_params, train, nfold=N_FOLDS, metrics=[EVAL_METRIC_XGB_REG],
                         early_stopping_rounds=100, stratified=False, seed=42)

        best_loss = cv_results['test-mae-mean'].iloc[-1] #or 'test-rmse-mean' if using RMSE

        return{'loss':best_loss, 'status': STATUS_OK }

    train = xgb.DMatrix(data, labels)

    #integer and string parameters, used with hp.choice()
    boosting_list = ['gbtree', 'gblinear'] #if including 'dart', make sure to set 'n_estimators'
    metric_list = ['mae'] 


    tree_method = [{'tree_method' : 'exact'},
           {'tree_method' : 'approx'},
           {'tree_method' : 'hist',
            'max_bin': hp.quniform('max_bin', 2**3, 2**7, 1),
            'grow_policy' : {'grow_policy': {'grow_policy':'depthwise'},
                            'grow_policy' : {'grow_policy':'lossguide',
                                              'max_leaves': hp.quniform('max_leaves', 32, XGB_MAX_LEAVES, 1)}}}]


    objective_list_reg = ['reg:linear', 'reg:gamma', 'reg:tweedie']
    objective_list_class = ['reg:logistic', 'binary:logistic']
    #for classification change line below to 'objective_list = objective_list_class'
    objective_list = objective_list_reg

    space ={'boosting' : hp.choice('boosting', boosting_list),
            'tree_method' : hp.choice('tree_method', tree_method),
            'max_depth': hp.quniform('max_depth', 2, XGB_MAX_DEPTH, 1),
            'reg_alpha' : hp.uniform('reg_alpha', 0, 5),
            'reg_lambda' : hp.uniform('reg_lambda', 0, 5),
            'min_child_weight' : hp.uniform('min_child_weight', 0, 5),
            'gamma' : hp.uniform('gamma', 0, 5),
            'learning_rate' : hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
            'eval_metric' : hp.choice('eval_metric', metric_list),
            'objective' : hp.choice('objective', objective_list),
            'colsample_bytree' : hp.quniform('colsample_bytree', 0.1, 1, 0.01),
            'colsample_bynode' : hp.quniform('colsample_bynode', 0.1, 1, 0.01),
            'colsample_bylevel' : hp.quniform('colsample_bylevel', 0.1, 1, 0.01),
            'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
            'nthread' : 3
        }

    trials = Trials()
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=num_evals, 
                trials=trials)

    best['tree_method'] = tree_method[best['tree_method']]['tree_method']
    best['boosting'] = boosting_list[best['boosting']]
    best['eval_metric'] = metric_list[best['eval_metric']]
    best['objective'] = objective_list[best['objective']]

    #cast floats of integer params to int
    for param in integer_params:
        best[param] = int(best[param])
    if 'max_leaves' in best:
        best['max_leaves'] = int(best['max_leaves'])
    if 'max_bin' in best:
        best['max_bin'] = int(best['max_bin'])

    print('{' + '\n'.join('{}: {}'.format(k, v) for k, v in best.items()) + '}')

    if diagnostic:
        return(best, trials)
    else:
        return(best)

In [ ]:
xgb_params = quick_hyperopt(X_train, y_train, 3000, diagnostic=False)

In [ ]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=0)

In [ ]:
def train_model(X, X_test, y, model_type, params=None, folds=folds, model=None):
    
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    
    for fold_n, (train_idx, valid_idx) in enumerate(folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_valid, y_valid = X.iloc[valid_idx], y.iloc[valid_idx]
        
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)
            model = xgb.train(dtrain=train_data, num_boost_round=20000,
                              evals=[(train_data, 'train'), (valid_data, 'valid_data')], 
                              early_stopping_rounds=200, verbose_eval=500, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns),
                                         ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns),
                                   ntree_limit=model.best_ntree_limit)
            
        if model_type == 'sklearn':
            model.fit(X_train, y_train)
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = mean_absolute_error(y_valid, y_pred_valid)
            print(f'Fold: {fold_n},  MAE: {score:.4f}')
            y_pred = model.predict(X_test).reshape(-1,)
            
        oof[valid_idx] = y_pred_valid
        scores.append(mean_absolute_error(y_valid, y_pred_valid))
        prediction += y_pred
        
    prediction /= n_fold
    print('CV mean score: {0:.4f}, std: {1:.4f}'.format(np.mean(scores), np.std(scores)))
    
    return oof, prediction

In [ ]:
oof_xgb, prediction_xgb = train_model(X_train, X_test, y_train, model_type='xgb', params=xgb_params, folds=folds, model=None)
pd.DataFrame(prediction_xgb, index=X_test.index, columns=['time_to_failure']).to_csv('submission_xgboost_hyperopt.csv', index=True, header=True)